# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [9]:
PUBLICATIONS = "publications_laura.tsv"

In [10]:
!cat PUBLICATIONS

cat: PUBLICATIONS: No such file or directory


## Import pandas

We are using the very handy pandas library for dataframes.

In [11]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [12]:
publications = pd.read_csv(PUBLICATIONS, sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,slides_url
0,2025-01-07,A Human Perspective to AI-based Candidate Scre...,58th Hawaii International Conference on System...,Skill extraction is at the core of algorithmic...,"Vásquez-Rodríguez, L., Audrin, B., Michel, S.,...",hicss,https://scholarspace.manoa.hawaii.edu/items/ed...,NaN
1,2024-10-14,Hardware-effective Approaches for Skill Extrac...,RecSys in HR’24: The 4th Workshop on Recommend...,Recent work on the automatic extraction of ski...,"Vásquez-Rodríguez, L., Audrin, B., Michel, S.,...",recsysinhr,https://recsyshr.aau.dk/wp-content/uploads/202...,https://docs.google.com/presentation/d/188QoZ0...
2,2023-12-08,BLESS: Benchmarking Large Language Models on S...,In Proceedings of the 2023 Conference on Empir...,"We present BLESS, a comprehensive performance ...","Kew, T., Chi, A., Vásquez-Rodríguez, L., Agraw...",bless,https://aclanthology.org/2023.emnlp-main.821/,NaN
3,2023-09-08,Document-level Text Simplification with Cohere...,In Proceedings of the Second Workshop on Text ...,We present a coherence-aware evaluation of doc...,"Vásquez-Rodríguez, L., Shardlow, M., Przybyła,...",coherence,https://aclanthology.org/2023.tsar-1.9/,NaN
4,2022-12-08,UoM and MMU @ TSAR2022-ST — PromptLS: Prompt L...,In Proceedings of the Workshop on Text Simplif...,"We present PromptLS, a method for fine-tuning ...","Vásquez-Rodríguez, L., Nguyen, N., Shardlow, M...",prompt-ls,https://aclanthology.org/2022.tsar-1.23/,https://aclanthology.org/2022.tsar-1.23.mp4
5,2022-12-08,A Benchmark for Neural Readability Assessment ...,In Proceedings of the Workshop on Text Simplif...,We release a new benchmark for Automated Reada...,"Vásquez-Rodríguez, L., Cuenca-Jiménez, P., Mor...",readability,https://aclanthology.org/2022.tsar-1.18/,https://www.youtube.com/watch?v=g-japt36HK0
6,2021-08-02,Investigating Text Simplification Evaluation,Evaluation. In Findings of the Association for...,Modern text simplification (TS) heavily relies...,"Vásquez-Rodríguez, L., Shardlow, M., Przybyła,...",investigating,https://aclanthology.org/2021.findings-acl.77/,https://aclanthology.org/2021.findings-acl.77.mp4
7,2021-09-21,The Role of Text Simplification Operations in ...,In Current Trends in Text Simplification (CTTS...,Research in Text Simplification (TS) has relie...,"Vásquez-Rodríguez, L., Shardlow, M., Przybyła,...",role,https://ceur-ws.org/Vol-2944/paper4.pdf,https://www.youtube.com/watch?v=fyGCS6q4Ljw
8,2019-05-19,PATHWAYS ANALYZER: Design of a Tool for the Sy...,BioRxiv. The preprint server for Biology.,The present work aims to design a tool for the...,"Vásquez-Rodríguez, L., Alvarado, R., Orozco, A...",pathways,https://www.biorxiv.org/content/10.1101/634998v3,NaN
9,2024-12-24,Simple is not Enough: Document-level Text Simp...,arXiv preprint.,We present a coherence-aware evaluation of doc...,"Vásquez-Rodríguez, L., Nguyen, N. T., Przybyła...",simple,https://arxiv.org/abs/2412.18655,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [13]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [14]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""  + "\n"
    md += """category: conferences"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.slides_url)) > 5:
        md += "\n[Download slides here](" + item.slides_url + ")\n" 

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [15]:
!ls ../_publications/

2019-05-19-pathways.md      2023-09-08-coherence.md
2021-08-02-investigating.md 2023-12-08-bless.md
2021-09-21-role.md          2024-10-14-recsysinhr.md
2022-12-08-prompt-ls.md     2024-12-24-simple.md
2022-12-08-readability.md   2025-01-07-hicss.md


In [16]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

cat: ../_publications/2009-10-01-paper-title-number-1.md: No such file or directory
